#**DATA UNDERSTANDING**

##**DATASET**

Source Dataset: https://www.kaggle.com/datasets/kekavigi/earthquakes-in-indonesia

In [3]:
import pandas as pd

data = "/content/katalog_gempa.csv"
df = pd.read_csv(data)

df

,tgl,ot,lat,lon,depth,mag,remark,strike1,dip1,rake1,strike2,dip2,rake2
0,2008/11/01,21:02:43.058,-9.18,119.06,10,4.9,Sumba Region - Indonesia,NaN,NaN,NaN,NaN,NaN,NaN
1,2008/11/01,20:58:50.248,-6.55,129.64,10,4.6,Banda Sea,NaN,NaN,NaN,NaN,NaN,NaN
2,2008/11/01,17:43:12.941,-7.01,106.63,121,3.7,Java - Indonesia,NaN,NaN,NaN,NaN,NaN,NaN
3,2008/11/01,16:24:14.755,-3.30,127.85,10,3.2,Seram - Indonesia,NaN,NaN,NaN,NaN,NaN,NaN
4,2008/11/01,16:20:37.327,-6.41,129.54,70,4.3,Banda Sea,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92882,2023/01/26,02:25:09.288,3.24,127.18,10,4.0,Talaud Islands - Indonesia,NaN,NaN,NaN,NaN,NaN,NaN
92883,2023/01/26,02:15:03.893,2.70,127.10,10,3.9,Northern Molucca Sea,NaN,NaN,NaN,NaN,NaN,NaN
92884,2023/01/26,01:57:08.885,-7.83,121.07,10,3.8,Flores Sea,NaN,NaN,NaN,NaN,NaN,NaN
92885,2023/01/26,01:46:21.009,3.00,127.16,10,4.1,Northern Molucca Sea,NaN,NaN,NaN,NaN,NaN,NaN


##**DATASET DESC**

Dataset ini terdiri dari **92.887 baris** dan **13 kolom** dalam format **.csv**, yang mencakup informasi seperti nomor kejadian, tanggal, waktu, garis bujur, garis lintang, magnitudo, daerah, serta kolom-kolom lain yang terkait dengan kejadian khusus. Data ini dirancang untuk mendukung upaya mitigasi bencana, pengembangan **sistem peringatan dini**, dan perencanaan **tata ruang yang lebih adaptif**. Dengan menganalisis pola dan karakteristik kejadian yang tercatat, dataset ini dapat menjadi landasan penting dalam mengambil langkah-langkah pencegahan dan penanggulangan bencana secara lebih efektif, serta membantu pemerintah dan pihak terkait dalam merancang kebijakan yang berbasis data untuk mengurangi risiko dan dampak bencana di masa depan.

#**DATA PREPARATION**

##**DATA CHECKING**

###Missing Values

In [4]:
print((df.isna().sum() / len(df)) * 100)

tgl         0.000000
ot          0.000000
lat         0.000000
lon         0.000000
depth       0.000000
mag         0.000000
remark      0.000000
strike1    97.055562
dip1       97.055562
rake1      97.055562
strike2    97.055562
dip2       97.055562
rake2      97.055562
dtype: float64


Terdapat 6 kolom yang Datanya missing lebih dari 70% yaitu: strike1, dip1, rake1, strike2, dip2, rake2.

###Duplicate

In [5]:
df[df.duplicated()]

,tgl,ot,lat,lon,depth,mag,remark,strike1,dip1,rake1,strike2,dip2,rake2


Tidak ada data yang terduplikasi pada dataset.

###Outliers

In [6]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

# Dataframe dari list hasil
results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

# Tampilkan dataframe
display(results_df)

Kolom,Persentase Outliers
lat,0.000000
lon,0.000000
depth,12.115797
mag,0.363883
strike1,0.000000
dip1,0.001077
rake1,0.000000
strike2,0.000000
dip2,0.000000
rake2,0.000000


Diketahui, nilai depth lebih dari 10% datanya yang berada pada outliers

###Inconsistent Values

Tidak ada inconsistent values pada dataset. kolom tanggal, waktu, dan kolom lainnya memiliki format yang sama

##**DATA CLEANING**

Beberapa permasalah pada data checking diantara lain:


*   Terdapat 6 kolom yang Datanya missing lebih dari 70% yaitu: strike1, dip1, rake1, strike2, dip2, rake2.
*   Nilai depth lebih dari 10% datanya yang berada pada outliers


Solusi permasalahan:


*   Menghapus 6 kolom yaitu: strike1, dip1, rake1, strike2, dip2, dan rake2. alasannya: 6 kolom tersebut memiliki data missing lebih dari 70% dan 6 kolom tersebut juga tidak relevan dengan studi kasus.
*  Nilai dept bisa diabaikan dikarenakan Outliers nya dibawah 10% dan outliersnya masih dalam kategori yang tidak dipermasalahkan.



In [7]:
#mengahapus kolom
df = df.drop(['strike1', 'dip1', 'rake1', 'strike2', 'dip2', 'rake2'], axis=1)
df

,tgl,ot,lat,lon,depth,mag,remark
0,2008/11/01,21:02:43.058,-9.18,119.06,10,4.9,Sumba Region - Indonesia
1,2008/11/01,20:58:50.248,-6.55,129.64,10,4.6,Banda Sea
2,2008/11/01,17:43:12.941,-7.01,106.63,121,3.7,Java - Indonesia
3,2008/11/01,16:24:14.755,-3.30,127.85,10,3.2,Seram - Indonesia
4,2008/11/01,16:20:37.327,-6.41,129.54,70,4.3,Banda Sea
...,...,...,...,...,...,...,...
92882,2023/01/26,02:25:09.288,3.24,127.18,10,4.0,Talaud Islands - Indonesia
92883,2023/01/26,02:15:03.893,2.70,127.10,10,3.9,Northern Molucca Sea
92884,2023/01/26,01:57:08.885,-7.83,121.07,10,3.8,Flores Sea
92885,2023/01/26,01:46:21.009,3.00,127.16,10,4.1,Northern Molucca Sea


##**DATA CONSTRUCTION**

Pada Data Construction saya ingin menambahkan sebuah kolom dengan menunjukan bahaya sebuah gempa berdasarkan magnitudo dengan indikator:

* 0-2.9: Mikrogempa
* 3-3.9: Gempa terasa ringan
* 4-4.9: Gempa terasa jelas
* 5-5.9: Gempa sedang
* 6-6.9: Gempa kuat
* 7-7.9: Gempa besar
* 8-8.9: Gempa sangat besar
* 9-10: Gempa dahsyat


In [8]:
def keterangan_gempa(magnitudo):
    if magnitudo < 3.0:
        return "Mikrogempa"
    elif 3.0 <= magnitudo < 4.0:
        return "Gempa terasa ringan"
    elif 4.0 <= magnitudo < 5.0:
        return "Gempa terasa jelas"
    elif 5.0 <= magnitudo < 6.0:
        return "Gempa sedang"
    elif 6.0 <= magnitudo < 7.0:
        return "Gempa kuat"
    elif 7.0 <= magnitudo < 8.0:
        return "Gempa besar"
    elif 8.0 <= magnitudo < 9.0:
        return "Gempa sangat besar"
    else:
        return "Gempa dahsyat"

df['Keterangan'] = df['mag'].apply(keterangan_gempa)

df

,tgl,ot,lat,lon,depth,mag,remark,Keterangan
0,2008/11/01,21:02:43.058,-9.18,119.06,10,4.9,Sumba Region - Indonesia,Gempa terasa jelas
1,2008/11/01,20:58:50.248,-6.55,129.64,10,4.6,Banda Sea,Gempa terasa jelas
2,2008/11/01,17:43:12.941,-7.01,106.63,121,3.7,Java - Indonesia,Gempa terasa ringan
3,2008/11/01,16:24:14.755,-3.30,127.85,10,3.2,Seram - Indonesia,Gempa terasa ringan
4,2008/11/01,16:20:37.327,-6.41,129.54,70,4.3,Banda Sea,Gempa terasa jelas
...,...,...,...,...,...,...,...,...
92882,2023/01/26,02:25:09.288,3.24,127.18,10,4.0,Talaud Islands - Indonesia,Gempa terasa jelas
92883,2023/01/26,02:15:03.893,2.70,127.10,10,3.9,Northern Molucca Sea,Gempa terasa ringan
92884,2023/01/26,01:57:08.885,-7.83,121.07,10,3.8,Flores Sea,Gempa terasa ringan
92885,2023/01/26,01:46:21.009,3.00,127.16,10,4.1,Northern Molucca Sea,Gempa terasa jelas


##**DATA REDUCTION**

Dataset diatas semua kolom masih relevan dengan studi kasus saya. dengan hasil akhir pada table berikut:

In [9]:
df

,tgl,ot,lat,lon,depth,mag,remark,Keterangan
0,2008/11/01,21:02:43.058,-9.18,119.06,10,4.9,Sumba Region - Indonesia,Gempa terasa jelas
1,2008/11/01,20:58:50.248,-6.55,129.64,10,4.6,Banda Sea,Gempa terasa jelas
2,2008/11/01,17:43:12.941,-7.01,106.63,121,3.7,Java - Indonesia,Gempa terasa ringan
3,2008/11/01,16:24:14.755,-3.30,127.85,10,3.2,Seram - Indonesia,Gempa terasa ringan
4,2008/11/01,16:20:37.327,-6.41,129.54,70,4.3,Banda Sea,Gempa terasa jelas
...,...,...,...,...,...,...,...,...
92882,2023/01/26,02:25:09.288,3.24,127.18,10,4.0,Talaud Islands - Indonesia,Gempa terasa jelas
92883,2023/01/26,02:15:03.893,2.70,127.10,10,3.9,Northern Molucca Sea,Gempa terasa ringan
92884,2023/01/26,01:57:08.885,-7.83,121.07,10,3.8,Flores Sea,Gempa terasa ringan
92885,2023/01/26,01:46:21.009,3.00,127.16,10,4.1,Northern Molucca Sea,Gempa terasa jelas
